In [ ]:
propno = 2671  # alt: 7076
runno = 208  # alt: 56
train_slice = slice(0, 100)

detector_id = "SPB_DET_AGIPD1M-1"
litframe_finder_id = "SPB_IRU_AGIPD1M1/REDU/LITFRM:output"

result_dir = f"proc/p{propno:06d}/r{runno:04d}"

litpx_counter_id = f"{detector_id}/REDU/LITPX_COUNTER"
spi_hitfinder_id = f"{detector_id}/REDU/SPI_HITFINDER"

adu_threshold = 0.7

modules=[3, 4, 8, 15]
mode="adaptive"
snr=4.0
min_scores=100
fixed_threshold=0
hitrate_window_size=200
xgm_norm=True
xgm_min=200
miss_fraction=1
miss_fraction_base="hit"

num_proc = 0

force_hitfinding = False

In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt
import multiprocessing as mp
import psutil

from extra_data import RunDirectory, open_run, PropertyNameError

from extra_redu.spi import LitPixelCounter, SpiHitfinder
from extra_redu.fileutils import PulseSource, StackedPulseSource, exdf_save
from extra_redu.litfrm import make_litframe_finder, LitFrameFinderError

from IPython.display import Latex, Markdown

In [ ]:
display(Markdown("# SPI Hit Finding"))

In [ ]:
try:
    if force_hitfinding:
        raise ValueError("force hitfinding")
    run = RunDirectory(result_dir)
    hitfinder = SpiHitfinder.from_datacollection(run, spi_hitfinder_id)
    do_hitfinding = False
    display(Markdown("- [x] read results from files\n"
                     "- [ ] analyse images"))
except (FileNotFoundError, KeyError, Exception):
    do_hitfinding = True
    display(Markdown("- [ ] read results from files\n"
                     "- [x] analyse images"))
    sources = {}

In [ ]:
if do_hitfinding:
    # 1. read litframe finder data
    dc = open_run(propno, runno, data="raw")
    litfrm = make_litframe_finder(detector_id[:3], dc, litframe_finder_id)
    try:
        #r = litfrm.read_or_process()
        r = litfrm.process()
        litfrm_src = PulseSource(
            r.meta.trainId, r.output.nFrame,
            r.output.detectorPulseId.ravel(),
            {
                "energyPerFrame": r.output.energyPerFrame.ravel(),
            }
        )
    except LitFrameFinderError:
        litfrm_src = None

In [ ]:
if do_hitfinding:
    # 2.1 read lit-pixel counts
    dc = open_run(propno, runno, data="proc")
    dc = dc.select_trains(train_slice)
    try:
        litpx_counter = StackedPulseSource.from_datacollection(
            dc, f"{detector_id}/CORR/(?P<key>\d+)CH0:output", "litpx")
        display(Markdown("- [x] read lit-pixel counts from files\n"))
    except (PropertyNameError, ValueError):
        litpx_counter = None
        

In [ ]:
if do_hitfinding and litpx_counter is None:
    # 2.2. count lit-pixels
    display(Markdown("- [x] run lit-pixel counter (takes time)\n"))
    
    dc = open_run(propno, runno, data="proc")
    dc = dc.select_trains(train_slice)

    src = StackedPulseSource.from_datacollection(
        dc, detector_id + r"/DET/(?P<key>\d+)CH0:xtdf", "image")

    litpx_counter = LitPixelCounter(src, threshold=adu_threshold)

    if not num_proc:
        num_proc = min(psutil.cpu_count(logical=False), 32)

    with mp.Pool(num_proc) as pool:
        chunks = src.split_trains(trains_per_part=1)
        results = pool.imap_unordered(litpx_counter, chunks)
        for _ in results:
            pass

    sources[litpx_counter_id] = litpx_counter

In [ ]:
if do_hitfinding:
    # 3. run hit finding
    hitfinder = SpiHitfinder(
        modules=modules,
        mode=mode,
        snr=snr,
        min_scores=min_scores,
        fixed_threshold=fixed_threshold,
        hitrate_window_size=hitrate_window_size,
        xgm_norm=xgm_norm,
        xgm_min=xgm_min,
        miss_fraction=miss_fraction,
        miss_fraction_base=miss_fraction_base,
    )
    hitfinder.find_hits(litpx_counter, litfrm_src)
    sources[spi_hitfinder_id] = hitfinder

In [ ]:
if do_hitfinding:
    # write results in file
    os.makedirs(result_dir, exist_ok=True)
    exdf_save(result_dir, "REDU00", runno, sources, sequence_size=3500)

In [ ]:
display(Markdown("## Hitrate"))
hitfinder.plot_hitrate()
plt.show()

In [ ]:
display(Markdown("## Hitscores histogram"))
hitfinder.plot_hitscore_hist()
plt.show()

In [ ]:
display(Markdown("## Hitscores over trains"))
hitfinder.plot_hitscore()
plt.show()